In [ ]:
import os

def is_float(value):
    try:
        float(value)
        return True
    except ValueError:
        return False

def is_yolo_format(line):           # Checking labels, whether they are in YOLO format
    parts = line.strip().split()
    if len(parts) != 5:
        return False
    if not all(is_float(part) for part in parts[1:]):
        return False
    return True
def check_yolo_format(label_dir):
    for filename in os.listdir(label_dir):
        if filename.endswith(".txt"):
            with open(os.path.join(label_dir, filename), 'r') as file:
                lines = file.readlines()
                for line in lines:
                    if not is_yolo_format(line):
                        print(f"Invalid YOLO format in {filename}: {line.strip()}")

label_directory = 'data/labels'
check_yolo_format(label_directory)


In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8n.yaml").load("yolov8n.pt")  # Build from YAML and transfer weights


# Tune hyperparameters (optionally)
# model.tune(data="data.yaml", epochs=5, iterations=20, optimizer="AdamW", plots=False, save=False, val=False)

In [ ]:
results = model.train(
    data='data.yaml',
   imgsz=1280,  
   epochs=100,
   batch=4,
   patience=7,
   amp=True,   
   optimizer='Adam',  # Adam optimizer
    lr0=0.001,
    momentum=0.9,
   val=True,
   resume=True, 
   name='yolov8n_trained',
   augment= True,       # Enable data aug while training
   hsv_h= 0.001,
   hsv_v= 0.1,   # Brightness augmentation
   hsv_s= 0.1,   # Saturation aug, for different environmental conditions
   shear= 0.01,   # For mimicking the effect of objects being viewed from different angles
   degrees= 15,
   crop_fraction= 0.2
   )

In [ ]:

model = YOLO('runs/detect/yolov8n_trained/weights/best.pt')     # Load the trained model
results = model.val(data='data.yaml', batch=4, imgsz=1280, split='test')    # Evaluate the model on the test set

print(results)

In [ ]:
import os
from ultralytics import YOLO

trained_model = YOLO("runs/detect/yolov8n_trained/weights/best.pt")

image_dir = r"Your\image\directory"      # Select the directory which the model make predictions on
output_dir = r"Your\output\directory"    # Select the directory which the model create the txt outputs to

os.makedirs(output_dir, exist_ok=True)

for image_name in os.listdir(image_dir):
    if image_name.endswith(('.jpg', '.jpeg', '.png', '.bmp')):  # Only image files
        image_path = os.path.join(image_dir, image_name)
        
        results = trained_model.predict(image_path, conf=0.25, iou=0.45)    # Apply NMS     

        for result in results:
            # Check if any detections were made
            if result.boxes.shape[0] > 0:  
                label_path = os.path.join(output_dir, os.path.splitext(image_name)[0] + '.txt')
                with open(label_path, 'w') as f:
                    for box in result.boxes:
                        try:
                            # Extract coordinates and class ID
                            class_id = int(box.cls.item())  
                            x_center, y_center, width, height = box.xywh[0].tolist()
                            
                            # Normalize coordinates
                            img_width, img_height = result.orig_shape[1], result.orig_shape[0] 
                            x_center_norm = x_center / img_width
                            y_center_norm = y_center / img_height
                            width_norm = width / img_width
                            height_norm = height / img_height

                            # Write to file in YOLO format
                            f.write(f"{class_id} {x_center_norm} {y_center_norm} {width_norm} {height_norm}\n")
                        except Exception as e:
                            print(f"Error processing box: {e}")
            else:
                print(f"No detections found for {image_name}, skipping file creation.")
